## Filtering Observed Arrivals
As the [API Exploration Notebook](API_exploration.ipynb) shows, each poll of the scraper produces 3 predicted arrival times for each line direction at a station. We want to transform and reduce these data to only feature observed train arrivals at stations ([per this issue](https://github.com/CivicTechTO/ttc_subway_times/issues/13)).

This notebook explores how to do this. 

In [23]:
from psycopg2 import connect
import configparser
import pandas as pd
import pandas.io.sql as pandasql

ImportError: No module named 'pandas'

In [2]:
CONFIG = configparser.ConfigParser(interpolation=None)
CONFIG.read('../db.cfg')
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

## Goal
We want to generate observed arrival times, in a format similar to [GTFS](https://github.com/google/transit/blob/master/gtfs/spec/en/reference.md#stop_timestxt). The GTFS schedule will be useful in this process, data was downloaded from [Transit Feeds](https://transitfeeds.com/p/ttc/33), the schema of the data is in [ttc_gtfs_create.sql](../ttc_gtfs_create.sql) and it is processed to a more useful format in PostgreSQL with [ttc_gtfs_process.sql](../ttc_gtfs_process.sql).

From gtfs, we can get a sense of the target data count, how many stops are scheduled on the three subway lines for which we have data?

In [8]:
sql = '''SELECT COUNT(1)

FROM gtfs.stop_times 
INNER JOIN gtfs.trips USING (trip_id)
INNER JOIN gtfs.routes USING (route_id)
INNER JOIN gtfs.calendar USING (service_id)
WHERE monday AND route_type = 1 AND route_short_name != '3'
'''

with con.cursor() as cur:
    cur.execute(sql)
    print(cur.fetchone()[0])

46404


This is a ball park figure we are aiming for in our filtering. 
Creating a materialized view of the raw poll data for a given day Wednesday, June 14th 2017 

In [17]:
sql = '''CREATE MATERIALIZED VIEW test_day AS 
SELECT requestid, stationid, lineid, create_date, request_date, station_char, subwayline, system_message_type, 
            timint, traindirection, trainid, train_message
FROM requests
INNER JOIN ntas_data USING (requestid)
WHERE request_date >= '2017-06-14'::DATE + interval '5 hours' 
AND request_date <  '2017-06-14'::DATE + interval '29 hours' 
''' 
with con:
    with con.cursor() as cur:
        cur.execute(sql)

In [18]:
with con.cursor() as cur:
    cur.execute('SELECT COUNT(1) FROM test_day')
    print(cur.fetchone()[0])

554390


Cool. Definitely some work to do.


Trying out a very basic filter, which has a [Known Issue](https://github.com/CivicTechTO/ttc_subway_times/issues/13#issuecomment-286609328)

In [22]:
sql = '''SELECT COUNT(DISTINCT (requestid, lineid, trainid, traindirection, stationid))
FROM test_day 
WHERE train_message = 'AtStation' OR timint < 1'''

with con.cursor() as cur:
    cur.execute(sql)
    print(cur.fetchone()[0])

81483


It's a start. 

If every line takes more than an hour to do a round-trip, we might be able to look for a distinct train-line-direction-station combination for each hour.

In [ ]:
sql = '''WITH trips AS (SELECT route_short_name, (SELECT trip_id FROM gtfs.trips WHERE trips.route_id = routes.route_id LIMIT 1) 
FROM gtfs.routes
WHERE route_type = 1 AND route_short_name != '3' )

SELECT route_id, trip_id, stop_id, arrival_time
FROM gtfs.stop_times
INNER JOIN trips USING(trip_id)
ORDER BY route_id, trip_id, stop_sequence'''

#load data into data frame

In [21]:
con.rollback()